# 전문가를 위한 파이썬

## part 3

## 챕터7

### 데커레이터

* 함수 데커레이터는 소스 코드에 있는 함수를 '표시'해서 함수의 작동을 개선할 수 있게 해준다. 강력한 기능이지만, 데커레이터를 자유자재로 사용하려면 먼저 클러저를 알아야 한다.
* 파이썬 3.0에 추가된 ```nonlocal```은 최근에 추가된 예약 키워드 중 하나다. 자기만의 데커레이터를 구현하고자 한다면 클로저를 속속들이 이해해야 하며, 그러고 나면 ```nonlocal```이 필요해진다.
* 데커레이터에서 사용하는 것 외에도, 클로저는 콜백을 이용한 효율적인 비동기 프로그래밍과 필요에 따라 함수형 스타일로 코딩하는 데에도 필수적이다.

### 데커레이터 기본 지식

* 데커레이터는 다른 함수를 인수로 받는 콜러블이다. 데커레이터는 데커레이트된 함수에 어떤 처리를 수행하고, 함수를 반환하거나 함수를 다른 함수나 콜러블 객체로 대체한다.

In [7]:
@decorate
def target():
    print('running target()')

In [8]:
def target():
    print('running target()')
target=decorate(target)

* 위 두 코드는 동일하게 작동한다.
* 데커레이터는 다름 함수를 인수로 전달해서 호출하는 일반적인 콜러블과 동일하다.
* 그렇지만 런타임에 프로그램 행위를 변경하는 "메타프로그래밍"을 할 때 데커레이터가 상당히 편리하다.

### 파이썬이 데커레이터를 실행하는 시점

* 데커레이터의 핵심 특징은 테커레이트된 함수가 정의된 직후에 실행된다는 것이다. 이는 일반적으로 파이썬 모듈을 로딩하는 시점, 즉 "임포트 타임"에 실행된다.

In [11]:
registry=[]

def register(func):
    print('running register(%s)' %func)
    registry.append(func)
    return func

@register
def f1():
    print('running f1()')
    
@register
def f2():
    print('running f2()')

def f3():
    print('running f3()')
    
def main():
    print('running main()')
    print('registry ->', registry)
    f1()
    f2()
    f3()

if __name__=='__main__':
    main()

running register(<function f1 at 0x00000186313EFB70>)
running register(<function f2 at 0x0000018631408048>)
running main()
registry -> [<function f1 at 0x00000186313EFB70>, <function f2 at 0x0000018631408048>]
running f1()
running f2()
running f3()


### 데커레이터로 개선한 전략 패턴

* 6.1절의 예제 6-6의 코드를 개선할 수 있다.

In [13]:
promos=[]

def promotion(promo_func):
    promos.append(promo_func)
    return promo_func

@promotion
def fidelity_promo(order):
    return order.total()*0.05 if order.customer.fidelity>=1000 else 0


@promotion
def bulkitem_promo(order):
    discount=0
    for item in order.cart:
      if item.quantity>=20:
          discount+=item.total()*.1
    return discount

@promotion
def largeorder_promo(self, order):
    distinct_items={item.product for item in order.cart}
    if len(distinct_items)>=10:
        return order.total*.07
    return 0

def best_promo(order):
    return max(promo(order) for promo in promos)

### 변수 범위 규칙

* 아래는 테스트 이다.

In [15]:
def f1(a):
    print(a)
    print(b)

f1(3)

3


NameError: name 'b' is not defined

In [16]:
b=6
f1(3)

3
6


* 당연하게도 b에 값을 할당하고 실행하면 정상적으로 작동한다.
* 하지만 다음 예제를 보자

In [18]:
b=6
def f2(a):
    print(a)
    print(b)
    b=9

f2(3)

3


UnboundLocalError: local variable 'b' referenced before assignment

* ???두번째 print문의 b가 출력되지 않고 error을 내는 것을 알 수 있다.
* 파이썬이 함수 본체를 컴파일할 때 b가 함수 안에서 할당되므로 b를 지역 변수로 판단한다.

### 간단한 데커레이터 구현하기

* 다음은 데커레이트된 함수를 호출할 때마다 시간을 측정해서 실행에 소요된 시간, 전달된 인수, 반환값을 출력하는 테커레이터다.

In [26]:
import time

def clock(func):
    def clocked(*args):
        t0=time.perf_counter()
        result=func(*args)
        elapsed=time.perf_counter()-t0
        name=func.__name__
        arg_str=', '.join(repr(arg) for arg in args)
        print('[%0.8fs] %s(%s) -> %r' % (elapsed, name, arg_str, result))
        return result
    return clocked

In [27]:
@clock
def snooze(seconds):
  time.sleep(seconds)

@clock
def factorial(n):
  return 1 if n<2 else n*factorial(n-1)

snooze(.123)
print(factorial(6))

[0.12385070s] snooze(0.123) -> None
[0.00000040s] factorial(1) -> 1
[0.00018970s] factorial(2) -> 2
[0.00040470s] factorial(3) -> 6
[0.00064060s] factorial(4) -> 24
[0.00086400s] factorial(5) -> 120
[0.00109540s] factorial(6) -> 720
720


### ```functools.lru_cache()```를 이용한 메모이제이션

* ```functools.lru_cache()```를 사용한 예제와 아닌 예제를 살펴보자.

In [29]:
@clock
def fibo(n):
  if n<2:
    return n
  return fibo(n-2)+fibo(n-1)

print(fibo(8))

[0.00000030s] fibo(0) -> 0
[0.00000030s] fibo(1) -> 1
[0.00009750s] fibo(2) -> 1
[0.00000030s] fibo(1) -> 1
[0.00000030s] fibo(0) -> 0
[0.00000020s] fibo(1) -> 1
[0.00002590s] fibo(2) -> 1
[0.00005180s] fibo(3) -> 2
[0.00017680s] fibo(4) -> 3
[0.00000020s] fibo(1) -> 1
[0.00000020s] fibo(0) -> 0
[0.00000020s] fibo(1) -> 1
[0.00002500s] fibo(2) -> 1
[0.00004940s] fibo(3) -> 2
[0.00000020s] fibo(0) -> 0
[0.00000020s] fibo(1) -> 1
[0.00002460s] fibo(2) -> 1
[0.00000020s] fibo(1) -> 1
[0.00000030s] fibo(0) -> 0
[0.00000020s] fibo(1) -> 1
[0.00002480s] fibo(2) -> 1
[0.00004940s] fibo(3) -> 2
[0.00009810s] fibo(4) -> 3
[0.00017190s] fibo(5) -> 5
[0.00037370s] fibo(6) -> 8
[0.00000020s] fibo(1) -> 1
[0.00000020s] fibo(0) -> 0
[0.00000020s] fibo(1) -> 1
[0.00002520s] fibo(2) -> 1
[0.00004980s] fibo(3) -> 2
[0.00000020s] fibo(0) -> 0
[0.00000020s] fibo(1) -> 1
[0.00002430s] fibo(2) -> 1
[0.00000020s] fibo(1) -> 1
[0.00000020s] fibo(0) -> 0
[0.00000020s] fibo(1) -> 1
[0.00002460s] fibo(2) -> 1
[

In [30]:
import functools

@functools.lru_cache()
@clock
def fibo(n):
  if n<2:
    return n
  return fibo(n-2)+fibo(n-1)

print(fibo(8))

[0.00000030s] fibo(0) -> 0
[0.00000050s] fibo(1) -> 1
[0.00017930s] fibo(2) -> 1
[0.00000070s] fibo(3) -> 2
[0.00021120s] fibo(4) -> 3
[0.00000060s] fibo(5) -> 5
[0.00024090s] fibo(6) -> 8
[0.00000050s] fibo(7) -> 13
[0.00027090s] fibo(8) -> 21
21


* 알고리즘을 처음 접할때는 메모이제이션이 쉽지 않은 개념이었는데 이런 방법도 있다니 흥미로웠다.

### 단일 디스패치를 이용한 범용 함수

* 파이썬 3.4에서 새로 소개된 ```functools.singledispatch()``` 데커레이터는 각 모듈이 전체 해결책에 기여할 수 있게 해주며, 편집할 수 없는 클래스에 대해서도 특화된 함수를 쉽게 제공할 수 있게 해준다.